### 1. 시험 내용 요약

1. PPT에서 12~13주차 강의자료 중 Part 1까지만 시험.  
    - Parsing the Log Record 부분은 시험에서 제외.  
    
    
2. Join > Sentiment Analysis > Rating 순으로 데이터를 처리하는 과정 시험 출제.  
    - 평점이 낮은 제품을 분석하는 과정  
        + EXPLODE 사용법  (Hive)
        + NGRAMS 사용법 (Hive)
        + SENTENCES 사용법 (Hive)
        + Hive query 작성법
        
        
3. Spark 관련 내용
    - .gloom() 사용법
    - .collect() 사용법
    - .getNumPartitions() 사용법
    - .paralellize() 사용법
    - .map() vs. .flatmap() 차이점 및 각각의 사용법
    - .filter() 사용법
    - .groupByKey() 사용법 및 데이터 처리 방식, 결과 예측
    - .reduceByKey() 사용법 및 데이터 처리 방식, 결과 예측
    - Word Count 과정
        + groupByKey를 사용한 경우
        + reduceByKey를 사용한 경우
    - 정규표현식

------


## 2. Hive를 이용한 구문 분석

#### Step 1. Rating을 이용해 평점이 낮은 제품을 선별

- Table 생성 및 데이터 로드
- 조건에 맞는 제품을 평점이 낮은 순으로 정렬


#### Step 2. NGRAMS를 이용해 평점이 낮은 제품의 상품평을 분석

- 문장을 단어 단위로 해체 (SENTENCES)
- 단어의 등장 빈도를 분석 (NGRAMS)
- Array(List)를 Records로 변환 (EXPLODE)
- 조건문과 LIKE를 이용한 상품평 추가 분석

------


## 3. PySpark

#### 1. SparkContext 시작

In [1]:
from pyspark import SparkConf as cf
from pyspark import SparkContext as ct

#### 사용자 정의 SparkContext
"제대로 작동 안 하는 듯. 사용하지 말것."


따로 시작하지 않아도 sc라는 이름으로 할당되어 있음.

In [7]:
usr_conf = (cf().setMaster("local").setAppName("Plentir").set("spark.executor.memory", "4g"))

# sc = cf(conf=usr_conf)

In [2]:
type(sc)

pyspark.context.SparkContext

In [5]:
sc.__dict__

{'_callsite': CallSite(function='<module>', file='C:\\Users\\plent\\AppData\\Local\\Continuum\\miniconda3\\lib\\site-packages\\IPython\\utils\\py3compat.py', linenum=188),
 'environment': {'PYTHONHASHSEED': '0'},
 '_conf': <pyspark.conf.SparkConf at 0x1d3c8768ec8>,
 '_batchSize': 0,
 '_unbatched_serializer': PickleSerializer(),
 'serializer': AutoBatchedSerializer(PickleSerializer()),
 'master': 'local[*]',
 'appName': 'PySparkShell',
 'sparkHome': None,
 '_jsc': JavaObject id=o14,
 '_accumulatorServer': <pyspark.accumulators.AccumulatorServer at 0x1d3c8768bc8>,
 '_javaAccumulator': JavaObject id=o17,
 '_encryption_enabled': False,
 'pythonExec': 'python',
 'pythonVer': '3.7',
 '_pickled_broadcast_vars': <pyspark.broadcast.BroadcastPickleRegistry at 0x1d3c8661468>,
 '_python_includes': [],
 '_temp_dir': 'C:\\Users\\plent\\AppData\\Local\\Temp\\spark-7ad38b94-d81e-4da1-a437-856d96b99764\\pyspark-8e529db6-4fa4-4f62-96fa-dfe6539da5bc',
 'profiler_collector': None}

In [9]:
sc.appName = "Plentir"
sc.appName

'Plentir'

#### 2. RDD 다루기


RDD 함수의 종류
- Actions  
RDD로부터 값을 반환한다.
    + count()
    + take(n): RDD의 첫 번째 요소부터 n개까지를 리스트로 반환한다.
    + collect(n): RDD의 모든 요소를 반환한다.
    + saveAsTextFile(path): 주어진 경로에 RDD를 텍스트 파일로 저장한다.


- Transformations  
입력으로 받은 RDD를 이용해 새로운(독립적인) RDD 객체를 정의한다.
    + map(func)
    + flatmap(func)
    + filter(func)
    + distinct(): 중복을 제거한다.
    + first(): RDD의 첫 번째 요소를 반환한다.
    + top(n): RDD의 가장 큰 요소 n개를 반환한다.  
Transformation 함수는 연속적으로 사용 가능하다.

RDD의 성질
- RDD는 불변하다(Immutable)
    + RDD에 있는 데이터는 수정할 수 없다.
    + 필요에 따라 데이터를 수정하는 시퀀스를 Transformation한다.
- RDD는 Lazy Execution하다.
    + RDD의 데이터는 Action 함수에 의한 작업이 실행될 때만 처리된다.

#### RDD 생성

In [26]:
raw = sc.textFile("./sample.txt", 4) # 1. 텍스트 파일에서 생성

In [23]:
raw.glom().collect()

[['Once more',
  'I summon you',
  'Out of the past',
  'With poignant love,',
  'You who nourished the poet'],
 ['And the lover.', 'I see your gary eyes', 'Looking out to sea'],
 ['In those Rockport summers,', 'Keeping a distance', 'Within the closeness'],
 ['Which was never intrusive', 'Opening out', 'Into the world.']]

In [25]:
raw.count()

14

In [29]:
raw = sc.textFile("./sample.txt")
raw_up = raw.map(lambda line: line.upper()) # 2. RDD에서 생성

raw_up.glom().collect()

[['ONCE MORE',
  'I SUMMON YOU',
  'OUT OF THE PAST',
  'WITH POIGNANT LOVE,',
  'YOU WHO NOURISHED THE POET',
  'AND THE LOVER.',
  'I SEE YOUR GARY EYES',
  'LOOKING OUT TO SEA'],
 ['IN THOSE ROCKPORT SUMMERS,',
  'KEEPING A DISTANCE',
  'WITHIN THE CLOSENESS',
  'WHICH WAS NEVER INTRUSIVE',
  'OPENING OUT',
  'INTO THE WORLD.']]

In [32]:
raw = sc.parallelize(range(1, 100 + 1, 3)) # 3. 메모리에 존재하는 데이터에서 생성
raw.glom().collect()

[[1, 4, 7, 10],
 [13, 16, 19, 22],
 [25, 28, 31, 34],
 [37, 40, 43, 46, 49],
 [52, 55, 58, 61],
 [64, 67, 70, 73],
 [76, 79, 82, 85],
 [88, 91, 94, 97, 100]]

#### 파티션

In [24]:
re_raw = raw.repartition(6) # 파티션 개수 변경
re_raw.glom().collect()

[[],
 ['Once more',
  'I summon you',
  'Out of the past',
  'With poignant love,',
  'You who nourished the poet'],
 ['Which was never intrusive', 'Opening out', 'Into the world.'],
 [],
 ['In those Rockport summers,', 'Keeping a distance', 'Within the closeness'],
 ['And the lover.', 'I see your gary eyes', 'Looking out to sea']]

In [21]:
re_raw.getNumPartitions()

6

#### RDD의 속성 확인

#### 3. Spark의 주요 함수

------

## 4. 정규표현식


#### 1. Symbols
- \w (모든 문자) / \W (문자를 제외한 모든 것)
- \d (모든 숫자) / \D (숫자를 제외한 모든 문자)
- \s (공백) / \S (공백을 제외한 모든 문자)
- \b (단어의 경계) / \B (단어의 경계아 아닌 모든 것 = 문자의 경계)
- ^ (문자열의 시작)
- $ (문자열의 끝)


#### 2. Expressions
- . (모든 문자, 숫자 및 공백 1개)


- [abc] (a, b, c 중 하나와 일치)
- [^abc] (a, b, c를 제외한 것과 일치)
- (abc) (abc라는 문자열을 하나의 덩어리(그룹)로 취급)
- first|second (first, second 중 한 쪽과 일치할 경우 매치. 단 둘을 모두 포함한 경우에는 매치되지 않음.)


- \* (0개 이상의 매치)
- \+ (1개 이상의 매치)
- ? (0개 ~ 1개의 매치)
- {a:b} ('a개 이상 ~ b개 이하'개의 매치)

#### 기초

In [9]:
import re

test = "MyBaat beated the Brat's boat."

In [10]:
res = re.match(r"at.", test)
print(res) # match는 아무것도 매치되지 않으면 None을 반환

None


In [12]:
res = re.match(r"Baat", test)
print(res) # .match()는 문자열의 시작부터 매치되는 가장 처음의 매치 결과를 반환한다.

None


In [13]:
res = re.match(r"[Bb][aert]at", test)
print(res) # .match()는 문자열의 시작부터 매치되는 가장 처음의 매치 결과를 반환한다.

None


In [16]:
res = re.search(r"Baat", test) # .search()는 문자열의 어디에서든 매치되는 가장 처음의 매치 결과를 반환한다.
res.group() # 매치 결과를 출력할 때는 .group() 함수를 사용해야 한다.

'Baat'

In [20]:
res = re.search(r"[Bb][ert]at", test)
res.group()

'beat'

In [24]:
res = re.findall(r"[Bb][aerto]at", test) # .finall()은 문자열의 어디에서든 매치되는 모든 매치 결과를 리스트로 반환한다.
res

['Baat', 'beat', 'Brat', 'boat']

#### Email 처리

In [37]:
email = "plentirssu@soongsil.ac.kr"

res = re.search(r"(?P<id>\w+)@(?P<host>\w+\.)+(ac\.kr|com|net|org|edu)", email) # ?P<name>으로 그룹에 이름을 정할 수 있음.
print(res.groups(), "\n")

print(res.group(0))
print(res.group(1))
print(res.group(2))
print(res.group(3), "\n")

print(res.group("host"))

('plentirssu', 'soongsil.', 'ac.kr') 

plentirssu@soongsil.ac.kr
plentirssu
soongsil.
ac.kr 

soongsil.


#### IP 처리

In [51]:
ip = "192.168.1.1 - -"

res = re.search(r"^(\S+) (\S+) (\S+)", ip)

print(res.group())
print(res.groups()) # 그룹이 없으면 빈 튜플을 출력

192.168.1.1 - -
('192.168.1.1', '-', '-')


#### 날짜 및 시간 처리

In [67]:
time = "[01/Aug/1995:13:53:01 -0400]"

res = re.search(r"\[([\w/]+):([\d:]+) (\-\d{4})\]", time)
print(res.groups())

print(res.group()) # PPT에서는 01/Aug...:01까지를 날짜로 취급

('01/Aug/1995', '13:53:01', '-0400')
[01/Aug/1995:13:53:01 -0400]


#### 메시지 처리

In [96]:
msg = "'GET /images/launch-logo.gif HTTP/1.0'"

res = re.search(r"'(\S+) (\S+) (\S+)'", msg)
#res
res.groups()

('GET', '/images/launch-logo.gif', 'HTTP/1.0')

#### 연습

In [118]:
string = """194.120.126.123, NL, Netherlands
94.126.119.173, FR, France
193.46.74.166, RU, Russian Federation
46.235.67.202, RU, Russian Federation
193.161.193.64, RU, Russian Federation"""

In [119]:
re.search(r"([^,\s]+), (\w+), (\w+)", string).groups()

('194.120.126.123', 'NL', 'Netherlands')

In [127]:
[re.search(r"([^,\s]+), (\w+), (\w+)", line).group(1) for line in string.split("\n") \
    if re.search(r"([^,\s]+), (\w+), (\w+)", line).group(2) != "FR"]

['194.120.126.123', '193.46.74.166', '46.235.67.202', '193.161.193.64']

------


## 5. Log file 처리하기 (=Part 1)
Chapter 4에서 했던 걸 하나로!

In [68]:
log = "127.0.0.1 - - [01/Aug/1995:00:00:01 -0400] 'GET /images/launch-logo.gif HTTP/1.0' 200"